#### 코로나 바이러스 영향을 중국인 관광객 증감분석

##### 외국인 출입국 통계 데이터 

한국관광 데이터랩 : https://datalab.visitkorea.or.kr/
- 제공하는 csv파일을 추가작업으로 년월일 xlsx로 변경


##### 엑셀데이터 로드
기준 2019 1월 데이터

In [42]:
# openpyxl 모듈 설치
!pip install openpyxl

In [43]:
# 데이터 분석용 모듈 임포트
import pandas as pd
import numpy as np

In [ ]:
# 2019년 1월 엑셀데이터 로드
# 전년동기 ~ 구성비까지의 컬럼은 필요없음
# 행 전년동기 ~ 구성비까지 내용은 스킵
kto_201901 = pd.read_excel('./raw/kto_201901.xlsx', header=1, usecols='A:G', skipfooter=4)

kto_201901

In [ ]:
kto_201901.tail()

#### 데이터 전처리

In [ ]:
# 데이터 수와 타입
kto_201901.info()

In [ ]:
# 행과 열값 리턴
kto_201901.describe()

In [ ]:
# 수치통계
kto_201901.describe()

In [ ]:
# 각 컬럼 0 부분이 필터링 (체크)
condition = (kto_201901['관광'] == 0) | \
            (kto_201901['상용'] == 0) | \
            (kto_201901['공용'] == 0) | \
            (kto_201901['유학/연수'] == 0) | \
            (kto_201901['기타'] == 0)
            
kto_201901_zero = kto_201901[condition]
kto_201901_zero

In [ ]:
# df에 기준년월 추가
kto_201901['기준년월'] = '2019-01'
kto_201901

In [ ]:
# 국적데이터만 67개 중
# unique()는 numpy.ndarray 변환
# 원래 67개, unique() 67개 (의미없음)
kto_201901['국적'].unique()

In [52]:
# 대륙 목록
contients_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']

contients_list

['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']

In [ ]:
kto_201901[kto_201901['국적'].isin(contients_list) == False]

In [ ]:
kto_201901_country = kto_201901[(kto_201901.국적.isin(contients_list) == False)]
# kto_201901_country.국적.unique()
kto_201901_country

In [ ]:
kto_201901_country.국적.unique()

In [ ]:
# 인덱스 재설정
kto_201901_country_newindex = kto_201901_country.reset_index(drop = True)
kto_201901_country_newindex.tail()

In [ ]:
# 대륙컬럼 다시 만들기
contients = ['아시아'] * 25 + ['아메리카'] * 5 + ['유럽'] * 23 + ['오세아니아'] * 3 + ['아프리카'] * 2 + ['기타'] + ['교포']

print(contients)

In [ ]:
kto_201901_country_newindex['대륙'] = contients
kto_201901_country_newindex.head()

In [ ]:
kto_201901_country_newindex['대륙'] = contients
kto_201901_country_newindex.tail()

In [ ]:
# 국적별 관광객 비율
# 관광, 상용, 공용, 유학/연수, 기타 중 관광
# kto_201901_country_newindex.관광 # 밑에랑 똑같음
kto_201901_country_newindex['관광객비율(%)'] = \
    round((kto_201901_country_newindex['관광'] / kto_201901_country_newindex['계']) * 100, 1) #소수점 첫째자리까지

kto_201901_country_newindex

In [71]:
# 분석 - 관광객 비율이 높은 국가와 낮은 국가
kto_201901_country_newindex.sort_values(by = '관광객비율(%)', ascending=False).head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3
55,대양주 기타,555,3,4,0,52,614,2019-01,오세아니아,90.4


In [75]:
# 대륙별 관광객 비율
kto_201901_country_newindex.pivot_table(values='관광객비율(%)', index='대륙', sort = False) # , aggfunc = 'mean',  sort = False는 대륙순서대로

,관광객비율(%)
대륙,
교포,0.000000
기타,61.100000
아메리카,68.200000
아시아,59.624000
아프리카,32.700000
오세아니아,84.833333
유럽,63.826087


In [76]:
# 중국 관광객만
kto_201901_country_newindex[(kto_201901_country_newindex['국적'] == '중국')]

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
17,중국,320113,2993,138,8793,60777,392814,2019-01,아시아,81.5


In [79]:
# 전체 외국인 관광객 수
tourist_sum = sum(kto_201901_country_newindex['관광'])
tourist_sum

884293

In [ ]:
# 전체 비율

kto_201901_country_newindex['전체비율(%)'] = \
    round((kto_201901_country_newindex['관광'] / tourist_sum ) * 100, 1)
kto_201901_country_newindex.head()

In [ ]:
# 관광객 전체비율이 높은 나라 내림차순 정렬
kto_201901_country_newindex.sort_values('전체비율(%)', ascending=False).head() # ascending=False 내림차순

##### 위 내용을 자동화할 수 있는 함수 만들기

125개 엑셀파일 전처리

In [ ]:
# 엑셀 파일 데이터프레임으로 재정리 함수

def create_kto_data(yyyy, mm):  # 년월 받아서 엑셀파일 오픈
    
    # 1. 엑셀파일 이름과 경로를 지정
    file_path = f'./raw/kto_{yyyy+mm}.xlsx'
    # print(file_path)
    
    # 2. 엑셀파일 불러오기
    df = pd.read_excel(file_path, header=1, usecols='A:G', skipfooter=4)
    
    # 3. 기준년월 컬럼 추가
    df['기준년월'] = f'{yyyy}-{mm}'
    
    # 4. 국적 컬럼에 대륙 제거 국가만 남기기
    contients_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']
    condition = df['국적'].isin(contients_list) == False
    df_country = df[condition].reset_index(drop = True)
    
    # 5. 대륙 컬럼을 추가
    contients = ['아시아'] * 25 + ['아메리카'] * 5 + ['유럽'] * 23 + ['오세아니아'] * 3 + ['아프리카'] * 2 + ['기타대륙'] + ['교포']
    
    df_country['대륙'] = contients
    
    # 6. 국가별 관광객비율(%) 컬럼추가
    df_country['관광객비율(%)'] = round((df_country['관광'] / df_country['계']) * 100, 1)
    
    # 7. 전체비율(%) 추가
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round((df_country['관광'] / tourist_sum ) * 100, 1)
    
    # 8. 결과 출력
    return df_country

create_kto_data('2020','05')

##### 반복문으로 125개 엑셀 하나로 합치기

In [ ]:
# kto_yyyymm.xlxs

df = pd.DataFrame()

for yyyy in range(2010, 2021): # 2020년까지니깐 +1 -> 2021을 넣어줘야 함
    for mm in range(1, 13):
        mm_str = str(mm).zfill(2)
        yyyymm = f'{yyyy}{mm_str}'
        if(yyyymm == '202006'): break
        temp = create_kto_data(str(yyyy), mm_str)

        print(yyyymm, temp.shape, sep='|')